In [5]:
import os
import pprint
from dotenv import load_dotenv
from OptiHPLCHandler import EmpowerHandler, EmpowerInstrumentMethod

In [ ]:
from applications.revert_method import revert_method

In [3]:
# get api address from .env file
load_dotenv('../.env_vars')
EMPOWER_API_ADDRESS = os.getenv('EMPOWER_API_ADDRESS')

In [4]:
# Create an instance of the EmpowerHandler class
handler = EmpowerHandler(project="WebAPI_test", address=EMPOWER_API_ADDRESS)

# Get the list of methods, select one, and get the method details
with handler:
    method_list = handler.GetMethodList()  # Get the list of instrument methods
    method_name = "Template_Method_1"
    full_method = handler.GetInstrumentMethod(method_name)

In [13]:
print(f"Method name: {full_method.method_name}")
print("-"*len(f"Method name: {full_method.method_name}"))
print(f"Column Temperature: {full_method.column_temperature}")
print("Gradient Table: ")
pprint.pprint(full_method.gradient_table)

Method name: Template_Method_1
------------------------------
Column Temperature: 45.0
Gradient Table: 
[{'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': 'Initial',
  'Flow': '0.300',
  'Time': 'Initial'},
 {'CompositionA': '40.0',
  'CompositionB': '60.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '10.00'},
 {'CompositionA': '10.0',
  'CompositionB': '90.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '10.10'},
 {'CompositionA': '10.0',
  'CompositionB': '90.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '12.10'},
 {'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '12.20'},
 {'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '20.00'}]


In [14]:
from types import SimpleNamespace

method = SimpleNamespace(
    method_name="Test",
    column_temperature=40,
    gradient_table=[{'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': 'Initial',
  'Flow': '0.300',
  'Time': 'Initial'},
 {'CompositionA': '40.0',
  'CompositionB': '60.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '10.00'},
 {'CompositionA': '10.0',
  'CompositionB': '90.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '10.10'},
 {'CompositionA': '10.0',
  'CompositionB': '90.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '12.10'},
 {'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '12.20'},
 {'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '20.00'}]
)

In [4]:
import sys
sys.path.append("../")
from applications.method_generators.rampup_method import generate_rampup_method 

generate_rampup_method(method=method, rampup_time=5, post_method=False, handler=None)
method.gradient_table

ModuleNotFoundError: No module named 'empower_implementation'